In [1]:
input_image_name = "Image_Image"
input_browser_name = "LandmarkingScan_1"
hide_model_name = "SkeletonModel"

In [2]:
# Switch to red-only layout

layoutManager = slicer.app.layoutManager()
layoutManager.setLayout(slicer.vtkMRMLLayoutNode.SlicerLayoutOneUpRedSliceView)

In [3]:
# Move input image off of any transforms, and delete all other images

input_image_node = slicer.util.getFirstNodeByName(input_image_name, className="vtkMRMLScalarVolumeNode")
if input_image_node is None:
    raise Exception("Cannot find image: {}".format(input_image_name))

input_image_node.SetAndObserveTransformNodeID(None)

volume_nodes = slicer.mrmlScene.GetNodesByClass('vtkMRMLScalarVolumeNode')
volume_nodes.InitTraversal()
volume = volume_nodes.GetNextItemAsObject()
while volume is not None:
    volume_name = volume.GetName()
    if volume_name != input_image_name:
        print("Delete {}".format(volume_name))
        slicer.mrmlScene.RemoveNode(volume)
    else:
        print("Keep   {}".format(volume_name))
    volume = volume_nodes.GetNextItemAsObject()


Delete Volume
Keep   Image_Image
Delete Image_Reference
Delete Segmented_Image
Delete LandmarkingScanReconstructionResults
Delete LandmarkingScan_1ReconstructionResults
Delete Volume_1
Delete Segmented_Image_1
Delete RoadmapScanReconstructionResults


In [4]:
# Show image in red view

redSliceWidget = slicer.app.layoutManager().sliceWidget('Red')
compNode = redSliceWidget.sliceLogic().GetSliceCompositeNode()
compNode.SetBackgroundVolumeID(input_image_node.GetID())
redSliceWidget.fitSliceToBackground()

redNode = slicer.mrmlScene.GetNodeByID('vtkMRMLSliceNodeRed')

resliceLogic = slicer.modules.volumereslicedriver.logic()
resliceLogic.SetDriverForSlice(input_image_node.GetID(), redNode)

In [5]:
# Delete all sequence browsers that are not the input

sequence_browsers = slicer.mrmlScene.GetNodesByClass('vtkMRMLSequenceBrowserNode')
sequence_browsers.InitTraversal()
browser = sequence_browsers.GetNextItemAsObject()
while browser is not None:
    browser_name = browser.GetName()
    if browser_name != input_browser_name:
        print("Delete {}".format(browser_name))
        slicer.mrmlScene.RemoveNode(browser)
    else:
        print("Keep   {}".format(browser_name))
    browser = sequence_browsers.GetNextItemAsObject()

Delete RoadmapScan
Delete LandmarkingScan
Keep   LandmarkingScan_1


In [6]:
# Delete all sequences that don't belong to the input browser

input_browser_node = slicer.util.getFirstNodeByName(input_browser_name)
if input_browser_node is None:
    raise Exception("Cannot find {}".format(input_browser_name))

sequence_nodes_to_keep = vtk.vtkCollection()
input_browser_node.GetSynchronizedSequenceNodes(sequence_nodes_to_keep, True)

sequences = slicer.mrmlScene.GetNodesByClass('vtkMRMLSequenceNode')
sequences.InitTraversal()
seq = sequences.GetNextItemAsObject()
while seq is not None:
    if sequence_nodes_to_keep.IsItemPresent(seq):
        print("Keep   {}".format(seq.GetName()))
    else:
        print("Delete {}".format(seq.GetName()))
        slicer.mrmlScene.RemoveNode(seq)
    seq = sequences.GetNextItemAsObject()

Delete RoadmapScan-TransdToReference-Seq
Delete RoadmapScan-Image_Image-Seq
Delete RoadmapScan-ImageToTransd-Seq
Delete LandmarkingScan-TransdToReference-Seq
Delete LandmarkingScan-Image_Image-Seq
Delete LandmarkingScan-ImageToTransd-Seq
Keep   LandmarkingScan_1-TransdToReference-Seq
Keep   LandmarkingScan_1-Image_Image-Seq
Keep   LandmarkingScan_1-ImageToTransd-Seq


In [7]:
# Create segmentation

segmentation_node = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode", "Segmentation")
segmentation_node.CreateDefaultDisplayNodes()
segmentation_node.SetReferenceImageGeometryParameterFromVolumeNode(input_image_node)
segmentation_node.GetSegmentation().AddEmptySegment()

'Segment_1'

In [8]:
# Create output browser

output_browser = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSequenceBrowserNode", "SegmentationBrowser")
sequence_browser_logic = slicer.modules.sequences.logic()

segmentation_sequence = sequence_browser_logic.AddSynchronizedNode(None, None, output_browser)
segmentation_sequence.SetName("SegmentationSequence")
sequence_browser_logic.AddSynchronizedNode(segmentation_sequence, segmentation_node, output_browser)
output_browser.SetRecording(segmentation_sequence, True)
output_browser.SetSaveChanges(segmentation_sequence, False)

image_sequence = sequence_browser_logic.AddSynchronizedNode(None, None, output_browser)
image_sequence.SetName("ImageSequence")
sequence_browser_logic.AddSynchronizedNode(image_sequence, input_image_node, output_browser)
output_browser.SetRecording(image_sequence, True)
output_browser.SetSaveChanges(image_sequence, False)

output_browser.SetRecordMasterOnly(True)

In [9]:
# Show sequence browser toolbar, and select input browser in it

sb_toolbar = slicer.modules.sequences.toolBar()
sb_toolbar.setVisible(True)

input_browser_node = slicer.util.getFirstNodeByName(input_browser_name, className="vtkMRMLSequenceBrowserNode")
slicer.modules.sequences.setToolBarActiveBrowserNode(input_browser_node)

In [10]:
# Hide model

hide_model_node = slicer.util.getFirstNodeByName(hide_model_name)
if hide_model_node is not None:
    hide_model_node.GetDisplayNode().SetVisibility(False)